In [1]:
import pandas as pd
import numpy as np

import os
import ntpath
import re
import time

from stat import *
from rdflib import *
from rdflib.namespace import *
from langdetect import detect
from datetime import datetime
from collections import Counter
from langdetect import DetectorFactory, detect  #to enforce consistent results

from scipy.stats import kstest
from scipy.stats import shapiro 

import random
import string
import csv
import re
import networkx as nx
import matplotlib.pyplot as plt

from langdetect.lang_detect_exception import LangDetectException

# Generating synthetic dataset

In [2]:
def generate_random_dataframe(n, m):
    data = {}
    for i in range(m):
        col_type = random.choice([int, float, str])
        if col_type == int:
            mean, std_dev = random.randint(0, 50), random.randint(1, 10)
            data[f'col_{i}'] = np.random.normal(mean, std_dev, n).astype(int)
            outliers = np.random.normal(mean * 3, std_dev * 3, int(n * 0.05)).astype(int)
            data[f'col_{i}'][:len(outliers)] = outliers
            print(len(outliers))
        elif col_type == float:
            mean, std_dev = random.uniform(0, 50), random.uniform(1, 10)
            data[f'col_{i}'] = np.random.normal(mean, std_dev, n)
            outliers = np.random.normal(mean * 3, std_dev * 3, int(n * 0.05))
            data[f'col_{i}'][:len(outliers)] = outliers
            print(len(outliers))
        elif col_type == str:
            data[f'col_{i}'] = [''.join(random.choices(string.ascii_letters, k=5)) for _ in range(n)]
    return pd.DataFrame(data)

# Uncheck the following to generate random dataset
df = generate_random_dataframe(250,5)
#df.to_csv("randomData5.csv",index=False)

12
12
12
12
12


In [3]:
def testNormality(columnValues):
    p_value = None
    if(len(columnValues) <= 50):
        _, p_value = shapiro(columnValues)
    else:
        _, p_value = (kstest(columnValues, 'norm'))
    if(p_value < 0.05):
        return False
    else:
        return True

In [4]:
pattern = re.compile(r'[^a-zA-Z0-9]') #removing non-alpha numeric characters from file name

In [5]:
#datatype RE
intType = re.compile(r"^\d+$")
dateType1 = re.compile(r"^\d{4}[-/]\d{1,2}[-/]\d{1,2}$")
dateType2 = re.compile(r"^\d{1,2}[-/]\d{1,2}[-/]\d{4}$")
stringType = re.compile("^[a-zA-Z]+.*\s*[a-zA-Z]*$")
floatType = re.compile(r"^[+-]?(\d*\.\d+|\d+\.?\d*)([eE][+-]?\d+)?$")
uriType = re.compile(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))")

In [6]:
#Predcting column datatype based on highest occurance of value
def typeCheck(singleCol):
    ci=cs=co=cf=cd=cu=0
    singleCol.fillna("$#", inplace = True)    #replace all NA with special characters
    for i in range(len(singleCol)):
        if((uriType.match(str(singleCol[i])))):
            cu+=1
        elif(stringType.match(str(singleCol[i]))):
            cs+=1
        elif((intType.match(str(singleCol[i]) ))):
            ci+=1
        elif(dateType1.match(str(singleCol[i]) or dateType2.match(str(singleCol[i])))):
            cd+=1
        elif(floatType.match(str(singleCol[i])) and singleCol[i]!='$#' ):
            cf+=1
        else:
            co+=1
    daConsidered=['int','str','float','date','uri','other']
    overall=[ci,cs,cf,cd,cu,co]
    di=zip(daConsidered, overall)
    #actDatatype=max(di)[0]
    if cf > ci :             #column with float values, int gets assigned to ci, coverting it to cf
        cf = cf+ci
        ci=0
    return overall.index(max(overall))

In [7]:
#Detecting language in case the column is string
def detectLang(singleCol):
    DetectorFactory.seed = 0
    lang = []   
    for r in singleCol:
        lang.append(detect(r))
    c=Counter(lang)
    totalItems=len(lang)
    return(c.most_common(1)[0][0],(c.most_common(1)[0][1]/totalItems)*100)

In [8]:
#No typos in date
def is_valid_date(year, month, day):
    day_count_for_month = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    if year%4==0 and (year%100 != 0 or year%400==0):
        day_count_for_month[2] = 29
    return (1 <= month <= 12 and 1 <= day <= int(day_count_for_month[month]))

In [9]:
#Count all NA's
def countNA(singleCol):
    resultedCounter = Counter(singleCol)
    return (resultedCounter[' ']+singleCol.isna().sum())

In [10]:
#Detecting valid date 
def checkDate(singleCol):
    year = pd.DatetimeIndex(singleCol).year  
    month = pd.DatetimeIndex(singleCol).month
    day = pd.DatetimeIndex(singleCol).day
    validDate = []
    for i in range(len(year)):
        validDate.append(is_valid_date(year[i], month[i], day[i]))
    trueCount = sum(validDate)
    if(trueCount == len(year)):
        return True
    else:
        return False

In [11]:
def identify_header(path, n=5, threshold=0.9):
    # Read n rows with and without a header
    df1 = pd.read_csv(path, header='infer', nrows=n)  # Assuming first row is the header
    df2 = pd.read_csv(path, header=None, nrows=n)     # Treating all rows as data
    
    # Get the first row from df2 (as data, not header) and align indexes with df1.dtypes
    first_row = df2.iloc[0]
    first_row.index = df1.dtypes.index  # Align indexes
    
    # Compare the data types of the first row with the inferred data types
    sim = (first_row.apply(type) == df1.dtypes).mean()  # Calculate similarity
    return sim < threshold

# Extract column names
def getSchema(csv_file):
    columnNames = []
    if identify_header(csv_file):  
        columnNames = list(pd.read_csv(csv_file, nrows=0).columns)  # Get header names
        has_header = 1
    else:
        # Generate column names like Attr1, Attr2, ...
        total_columns = pd.read_csv(csv_file, header=None, nrows=1).shape[1]
        columnNames = [f"Attr{i+1}" for i in range(total_columns)]
        has_header = 0
    return has_header, columnNames

In [12]:
#Calculating mean and std
def truncate(n, decimals=0):
    multiplier = 10 ** decimals
    return int(n * multiplier) / multiplier
def meanStd(columnValue):
    return truncate(np.mean(columnValue),2),truncate(np.std(columnValue),2)

In [13]:
import os
print(os.getcwd())

/home/d19125691/Experiments/Experiments/csv2rdf/csvtordfgit/csvdataqualityassessment/preprocessing


In [14]:
CSVW = Namespace("http://www.w3.org/ns/csvw#")
CSVTORDF = Namespace("https://purl.archive.org/domain/csvtordf#")

g = Graph()

#ontology file location
g.parse("./csvtordf.owl")
g.bind("csvw", CSVW)
g.bind("csvtordf", CSVTORDF)

In [15]:
#csv_file = "/home/d19125691/Experiments/Experiments/csv2rdf/csv2rdfworking/datasets/education.csv"
csv_file = "/home/d19125691/Experiments/Experiments/csv2rdf/csvtordfgit/csvdataqualityassessment/datasets/iris.csv"

nocolumnheader = 0

df = pd.read_csv(csv_file)
nocolumnheader, columnNames = getSchema(csv_file)

cleanedNames = []
for cN in columnNames:
    cleanedNames.append(pattern.sub('', cN))

columnNames = cleanedNames

with open(csv_file, mode='r') as file:
    reader = csv.reader(file)
    if(nocolumnheader):
        header = next(reader)
    rows = [row for row in reader] 
    for i, row in enumerate(reader):
        rows.append(row)
file_name = os.path.basename(csv_file)

fileName = os.path.splitext(file_name)[0]

In [16]:
#efine the file path
#csv_file = "/home/d19125691/Experiments/Experiments/csv2rdf/csv2rdfworking/preprocessing/iris_sample.csv"

# Initialize variables
#nocolumnheader = 0
#
#df = pd.read_csv(csv_file)

# getSchema provides the schema of the file
#nocolumnheader, columnNames = getSchema(csv_file)

# Clean column name
#cleanedNames = [re.sub(r'\W+', '', cN) for cN in columnNames]
#columnNames = cleanedNames

#rows = []
#with open(csv_file, mode='r') as file:
 #   reader = csv.reader(file)
  #  if nocolumnheader:
   #     print("NO")
    #    header = next(reader)
    #for i, row in enumerate(reader):
     #   rows.append(row)

#file_name = os.path.basename(csv_file)
#fileName = file_name[0]  # Extract the first character of the file name

In [17]:
count=0   #count triples
flag=0
countlongURI = 0 #counting Long URI if any 
count=0   #count triples
completeness = 0  #population completeness in the file
completeFlag = 0

completeDataset = 0
totalmissingvalues = df.isna().sum().sum() / (df.shape[0]*df.shape[1])

table_uri = CSVW.Table+fileName
completeness_uri = CSVW.Completeness

g.add((table_uri, RDF.type, CSVW.Table))

for i in range(df.shape[1]):
    flag=0
    column_uri = CSVW.Column+"_"+columnNames[i]
    completeness = countNA(df.iloc[:,i])
    if(completeness):
        constraint_node = BNode()
        g.add((constraint_node, RDF.type, CSVTORDF.DataQuality))
        g.add((column_uri, CSVTORDF.hasConstraint, constraint_node))
        g.add((constraint_node, CSVTORDF.constraintType, completeness_uri))
        g.add((constraint_node, CSVTORDF.constraintElement, Literal(completeness, datatype=XSD.float)))

    colDT = typeCheck(df.iloc[:, i])
    if(colDT==0):
        datatype = XSD.integer
        mean,std = meanStd(df.iloc[:, i])
        flag=1
    elif(colDT==1):
        flag=2
        datatype = XSD.string
    elif(colDT==2):
        datatype = XSD.float
        mean,std = meanStd(df.iloc[:, i])
        flag = 1
    elif(colDT==3):
        datatype = XSD.dateTime
        flag=3
    elif(colDT==4):
        datatype=XSD.anyURI
        flag=4
    else:
        flag = 4
        datatype = XSD.string
    
    g.add((column_uri, RDF.type, CSVW.Column))
    g.add((column_uri, CSVW.datatype, datatype))
    print(column_uri, datatype)
    count=count+2   
    
    if(flag==1):
        result = testNormality(df.iloc[:,i])
        if(result):
            g.add((column_uri, CSVTORDF.mean, Literal(mean, datatype=XSD.float) ))
            g.add((column_uri, CSVTORDF.stdDev, Literal(std, datatype=XSD.float) ))
        else:
            Q1 = np.percentile(df.iloc[:,i], 25)
            Q3 = np.percentile(df.iloc[:,i], 75)
            g.add((column_uri, CSVTORDF.Q1, Literal(Q1, datatype=XSD.float) ))
            g.add((column_uri, CSVTORDF.Q3, Literal(Q3, datatype=XSD.float) ))
        count+=2
    if(flag==2 and completeFlag != 1):
        try:
            lang, percentage = detectLang(df.iloc[:,i])
            g.add((column_uri, CSVTORDF.lang, Literal(lang)))
        except LangDetectException:
            print("Unable to detect language")
        count+=1
    if(flag==3):
        validDate = checkDate(df.iloc[:,i])
        if(validDate != True):
            g.add((column_uri, CSVTORDF.invalidDate, Literal(True, datatype=XSD.boolean)))
            count+=1

http://www.w3.org/ns/csvw#Column_0 http://www.w3.org/2001/XMLSchema#float
http://www.w3.org/ns/csvw#Column_1 http://www.w3.org/2001/XMLSchema#float
http://www.w3.org/ns/csvw#Column_2 http://www.w3.org/2001/XMLSchema#float
http://www.w3.org/ns/csvw#Column_3 http://www.w3.org/2001/XMLSchema#float
http://www.w3.org/ns/csvw#Column_4 http://www.w3.org/2001/XMLSchema#string


In [18]:
for rownum, row in enumerate(rows, start=1):
    row_uri = CSVW.Row+f"={rownum}"
    g.add((row_uri, RDF.type, CSVW.Row))
    g.add((table_uri, CSVW.row, row_uri))
    g.add((row_uri, CSVW.rownum, Literal(rownum, datatype=XSD.int)))
    for colnum, value in enumerate(row, start=1):
        cell_uri = CSVW.Cell+f"={rownum}-"+columnNames[colnum-1]
        column_uri = CSVW.Column+"_"+columnNames[colnum-1]
        g.add((cell_uri, RDF.type, CSVW.Cell))
        g.add((row_uri, CSVTORDF.hasCell, cell_uri))
        g.add((column_uri, RDF.type, CSVW.Column))
       # g.add((row_uri, CSVW.describes, column_uri))
        g.add((cell_uri, CSVTORDF.belongsToColumn, column_uri))
        if(value == ""):
            count=count+1
        elif(intType.match(str(value))):   
            g.add((cell_uri, CSVTORDF.hasValue, Literal(value, datatype=XSD.integer)))
            count=count+1
        elif((dateType1.match(str(value))) or (dateType2 .match(str(value)))):
            g.add((cell_uri, CSVTORDF.hasValue, Literal(value, datatype=XSD.datetime)))
            count=count+1
        elif((uriType.match(str(value)))):
            g.add((cell_uri, CSVTORDF.hasValue, Literal(value, datatype=XSD.anyURI)))
            count=count+1
        elif((stringType.match(str(value)))):
            g.add((cell_uri, CSVTORDF.hasValue, Literal(value, datatype=XSD.string)))
            count=count+1
        elif((floatType.match(str(value)))):
            g.add((cell_uri, CSVTORDF.hasValue, Literal(value, datatype=XSD.float)))
            count=count+1
        else:
            g.add((cell_uri, CSVTORDF.hasValue, Literal(value, datatype=XSD.string)))
            count=count+1

In [19]:
print(table_uri)

http://www.w3.org/ns/csvw#Tableiris


In [20]:
g.serialize("./TTL files/iris.ttl", format="ttl")

In [21]:
from owlready2 import *
filetoread = "file://"+os. getcwd()+"/TTL files/randomData2.ttl"
onto = get_ontology(filetoread).load()

#with onto:
#    Imp().set_as_rule("Table(?table), Row(?row1), row(?table, ?row1), Column(?col), Cell(?cell), describes(?row1, ?col), contains(?col, ?cell), datatype(?cell, ?a), ")

sync_reasoner_pellet([onto], infer_data_property_values=True)

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation
* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /home/d19125691/.local/lib/python3.6/site-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/home/d19125691/.local/lib/python3.6/site-packages/owlready2/pellet/jgrapht-jdk1.5.jar:/home/d19125691/.local/lib/python3.6/site-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/home/d19125691/.local/lib/python3.6/site-packages/owlready2/pellet/xml-apis-1.4.01.jar:/home/d19125691/.local/lib/python3.6/site-packages/owlready2/pellet/httpclient-4.2.3.jar:/home/d19125691/.local/lib/python3.6/site-packages/owlready2/pellet/commons-codec-1.6.jar:/home/d19125691/.local/lib/python3.6/site-packages/owlready2/pellet/antlr-3.2.jar:/home/d19125691/.local/lib/python3.6/site-packages/owlready2/pellet/jena-core-2.10.0.jar:/home/d19125691/.local/lib/python3.6/site-packages/owlready2/pellet/jena-arq-2.10.0.jar:/home/d19125

In [22]:
sync_reasoner([onto])

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /home/d19125691/.local/lib/python3.6/site-packages/owlready2/hermit:/home/d19125691/.local/lib/python3.6/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////tmp/tmpje431t5a
* Owlready2 * HermiT took 0.3672597408294678 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)
